# GPT3.5 Round Table

In [1]:
%cd /content/drive/MyDrive/NLP/semevaltask9

/content/drive/MyDrive/NLP/semevaltask9


In [2]:
import torch
import numpy as np
import pandas as pd
import random
from torch.utils.data import DataLoader, Dataset
from sklearn.metrics import accuracy_score

In [3]:
GLOBAL_SEED = 255

np.random.seed(GLOBAL_SEED)
random.seed(GLOBAL_SEED)
torch.manual_seed(GLOBAL_SEED)
torch.use_deterministic_algorithms(True)
%env CUBLAS_WORKSPACE_CONFIG=:4096:8

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


env: CUBLAS_WORKSPACE_CONFIG=:4096:8


### Load datasets

In [4]:
# Import Dataset
data = np.load("./Data/SP_train.npy", allow_pickle = True ).tolist()
SP_train = pd.DataFrame(data)
data = np.load("./Data/SP_test.npy", allow_pickle = True ).tolist()
SP_test = pd.DataFrame(data)
data = np.load("./Data/SP_test_answer.npy", allow_pickle = True ).tolist()
SP_test_answer = pd.DataFrame(data)

data = np.load("./Data/WP_train.npy", allow_pickle = True ).tolist()
WP_train = pd.DataFrame(data)
data = np.load("./Data/WP_test.npy", allow_pickle = True ).tolist()
WP_test = pd.DataFrame(data)
data = np.load("./Data/WP_test_answer.npy", allow_pickle = True ).tolist()
WP_test_answer = pd.DataFrame(data)

data = np.load("./Data/sentence_puzzle.npy", allow_pickle = True ).tolist()
SP = pd.DataFrame(data)
data = np.load("./Data/word_puzzle.npy", allow_pickle = True ).tolist()
WP = pd.DataFrame(data)

In [5]:
for index, row in SP_test.iterrows():
  SP_test.at[index, 'label'] = int(SP_test_answer.loc[index][1])

In [6]:
for index, row in WP_test.iterrows():
  WP_test.at[index, 'label'] = int(WP_test_answer.loc[index][1])

## Initial Round

### Create DataFrame

In [ ]:
gptResult = []
count = 1
with open("/content/init_answers.txt") as file_in:
    lines = []
    for line in file_in:
        if line.split('.')[0] == str(count):
          Solution = line.split("Solution: ")[1][0]
          count +=1
        elif "Reasoning:" in line:
          Reasoning = line.split('Reasoning: ')[1].split('\n')[0]
        elif "Confidence: " in line:
          Confidence = line.split('Confidence: ')[1].split('\n')[0]
          gptResult.append({"Solution" : Solution, "Reasoning" : Reasoning, "Confidence" : Confidence})


In [ ]:
gptResult

[{'Solution': 'A',
  'Reasoning': 'The relationship between the hardworking farmer and the lazy farmer seems unusual because typically, the father is the one associated with farming. However, in this scenario, the lazy farmer is the mother of the hardworking farmer, which explains why the hardworking farmer is working in the fields while his lazy mother is not. ',
  'Confidence': '1.0'},
 {'Solution': 'C',
  'Reasoning': "Given the description, it seems unlikely that glass shards and water on the floor would result from a shelf simply moving and crushing them. The scenario is more plausible if Romeo and Juliet were fish in a tank that fell due to the train's vibrations, causing the glass to break and them to perish.",
  'Confidence': '0.9'},
 {'Solution': 'B',
  'Reasoning': "In the year before someone's birthday, they are one year younger than they will be tomorrow. This typically happens during the first year of graduation when someone completes their academic year but hasn't yet cel

In [ ]:
gptResultDF = pd.DataFrame(gptResult)
gptResultDF

,Solution,Reasoning,Confidence
0,A,The relationship between the hardworking farme...,1.0
1,C,"Given the description, it seems unlikely that ...",0.9
2,B,"In the year before someone's birthday, they ar...",0.8
3,B,"In sports organizations, if an assistant capta...",1.0
4,C,If 5% of the population has an unlisted phone ...,1.0
...,...,...,...
115,B,Since the fisherman catches three more fish ea...,1.0
116,C,Bob's inability to find anything on Google was...,1.0
117,B,Similar to the riddle about marrying many wome...,1.0
118,B,The question asks for the number of legs on th...,1.0


In [ ]:
gptResultDF.to_csv('./GPT3.5_R0.csv')

### Round 1

In [ ]:
# import all round 0 answers:
gpt = pd.read_csv("./GPT3.5_R0.csv")
mixtral = pd.read_csv("./Mixtral_R0.csv")
claude = pd.read_csv("./Claude_R0.csv")

In [ ]:
mixtral

,Unnamed: 0.1,Unnamed: 0,Solution,Reasoning,Confidence
0,0,0,A,Let us analyze the given information:\n1. Ther...,0.70
1,1,1,C,"At first glance, it appears that Romeo and Jul...",0.90
2,2,2,C,"Before your birthday each year, you are always...",1.00
3,3,3,B,"Generally speaking, in sports organizations, t...",0.90
4,4,4,C,We know that 5% of the population has an unlis...,0.95
...,...,...,...,...,...
115,115,115,C,To calculate the number of fish caught on the ...,0.95
116,116,116,C,"When the Internet connection is disrupted, use...",0.95
117,117,117,B,"Given the statement, the person in considerati...",0.90
118,118,118,D,We can analyze this question by calculating th...,0.95


In [ ]:
prompt = """Please answer the question with step-by-step reasoning. Also,evaluate your confidence level (between 0.0 and 1.0) to indicate the possibility of your answer being right. Please give the answer in this template:
Solution:
Reasoning:
Confidence:

Question: """

def initial_prompt(item, prompt):

  labels = ["A", "B", "C", "D"]
  prompt += f'{item["question"]}\n'
  for index, choice in enumerate(item["choice_list"]):
    prompt += f'{labels[index]} : {choice}\n'
  return prompt

def generate_prompt(gpt, mixtral, claude, prompt):
  prompts = []

  for index in range(120):
    basePrompt = initial_prompt(SP_test.loc[index], prompt)
    basePrompt += """\nCarefully review the following solutions from other agents as additional
information, and provide your own answer and step-by-step reasoning to the question.

Clearly state which point of view you agree or disagree with and why.\n\n"""

    basePrompt += f"Agent one thinks the Answer is {claude.loc[index]['Solution']}.\nagent one reason: {claude.loc[index]['Reasoning']}\nagent one confidence: {claude.loc[index]['Confidence']}\n"
    basePrompt += f"Agent two thinks the Answer is {mixtral.loc[index]['Solution']}.\nagent two reason: {mixtral.loc[index]['Reasoning']}\nagent two confidence: {mixtral.loc[index]['Confidence']}\n"
    basePrompt += f"Agent three thinks the Answer is {gpt.loc[index]['Solution']}.\nagent three reason: {gpt.loc[index]['Reasoning']}\nagent three confidence: {gpt.loc[index]['Confidence']}\n"
    prompts.append(basePrompt)

  return prompts


In [ ]:
prompts = generate_prompt(gpt, mixtral, claude, prompt)

In [ ]:
for index, pr in enumerate(prompts):
  print(index + 1)
  print(pr)
  print("##################################################################################")

1
Please answer the question with step-by-step reasoning. Also,evaluate your confidence level (between 0.0 and 1.0) to indicate the possibility of your answer being right. Please give the answer in this template:
Solution:
Reasoning:
Confidence:

Question: In a small village, two farmers are working in their fields - a diligent farmer and a lazy farmer. The hardworking farmer is the son of the lazy farmer, but the lazy farmer is not the father of the hardworking farmer. Can you explain this unusual relationship?
A : The lazy farmer is his mother.
B : The lazy farmer is not a responsible father as he is lazy.
C : The diligent farmer devoted himself to the farm and gradually forgot his father.
D : None of above.

Carefully review the following solutions from other agents as additional
information, and provide your own answer and step-by-step reasoning to the question.

Clearly state which point of view you agree or disagree with and why.

Agent one thinks the Answer is A.
agent one reaso



### Make round1 dataframe

In [ ]:
gptResult = []
count = 1
with open("/content/r1_answers.txt") as file_in:
    lines = []
    for line in file_in:
        if "Solution:" in line:
          if "The answer is" in line:
            Solution = line.split("The answer is ")[1][0]
          else:
            Solution = line.split("Solution: ")[1][0]
          count +=1
        elif "Reasoning:" in line:
          Reasoning = line.split('Reasoning: ')[1].split('\n')[0]
        elif "Confidence: " in line:
          Confidence = line.split('Confidence: ')[1].split('\n')[0]
          gptResult.append({"Solution" : Solution, "Reasoning" : Reasoning, "Confidence" : Confidence})


In [ ]:
gptResultDF = pd.DataFrame(gptResult)
gptResultDF

,Solution,Reasoning,Confidence
0,A,The unusual relationship between the hardworki...,1.0
1,C,Given the presence of glass shards and water o...,1.0
2,C,"Before someone's birthday, they are always one...",1.0
3,B,"If a crucial assistant captain departs, the pr...",1.0
4,C,If 5% of the population has an unlisted phone ...,1.0
...,...,...,...
115,B,If the fisherman caught 30 fish in total over ...,1.0
116,C,Bob couldn't find anything on Google because h...,1.0
117,B,A preacher can marry others through wedding ce...,1.0
118,B,Only the bed's legs and the person's legs coun...,1.0


In [ ]:
gptResultDF.to_csv('./GPT3.5_R1.csv')

In [ ]:
mapping = {'A' : 0, 'B' : 1, 'C' : 2, 'D' : 3}

for index, item in enumerate(gptResultDF['Solution'].tolist()):
  print(mapping[item])

0
2
2
1
2
0
0
1
0
1
1
0
2
0
1
1
0
2
2
1
0
2
1
0
2
2
0
2
0
2
0
2
2
2
0
2
1
0
2
2
1
1
0
0
1
0
2
2
0
0
0
0
2
1
1
0
0
1
1
3
0
1
1
1
1
2
2
1
2
0
2
0
1
2
1
2
0
1
1
0
3
0
1
0
3
1
2
1
0
2
1
2
3
0
2
1
1
2
0
1
1
1
0
2
2
2
1
1
2
1
1
1
0
2
0
1
2
1
1
2


### Make round 2 Dataframe

In [7]:
gptResult = []
count = 1
with open("/content/r2_answers.txt") as file_in:
    lines = []
    for line in file_in:
        if "Solution:" in line:
          if "The answer is" in line:
            Solution = line.split("The answer is ")[1][0]
          else:
            Solution = line.split("Solution: ")[1][0]
          count +=1
        elif "Reasoning:" in line:
          Reasoning = line.split('Reasoning: ')[1].split('\n')[0]
        elif "Confidence: " in line:
          Confidence = line.split('Confidence: ')[1].split('\n')[0]
          gptResult.append({"Solution" : Solution, "Reasoning" : Reasoning, "Confidence" : Confidence})


In [8]:
gptResultDF = pd.DataFrame(gptResult)
gptResultDF

,Solution,Reasoning,Confidence
0,A,Given that the hardworking farmer is the son o...,1.0
1,C,"The presence of glass shards and water, along ...",1.0
2,C,"Before someone's birthday, they are always one...",1.0
3,B,"If a crucial assistant captain departs, the pr...",1.0
4,C,If 5% of the population has an unlisted phone ...,1.0
...,...,...,...
115,D,"Upon revisiting the problem, it seems there wa...",0.9
116,C,Bob's inability to find anything on Google is ...,1.0
117,B,A preacher officiates marriages but may not ha...,1.0
118,B,The question asks for the number of legs on th...,1.0


In [9]:
gptResultDF.to_csv('./GPT3.5_R2.csv')

In [10]:
mapping = {'A' : 0, 'B' : 1, 'C' : 2, 'D' : 3}

for index, item in enumerate(gptResultDF['Solution'].tolist()):
  print(mapping[item])

0
2
2
1
2
0
0
1
0
1
1
0
2
0
1
1
0
2
2
0
0
0
1
0
0
0
0
2
0
2
1
2
2
2
2
2
1
2
2
2
1
1
0
0
1
0
2
2
0
0
0
0
1
0
1
0
0
1
1
3
0
1
1
1
1
0
2
1
2
0
2
0
3
2
1
2
0
1
1
0
3
0
1
0
3
1
2
3
1
2
3
2
3
0
2
1
1
2
0
1
1
2
0
2
2
2
3
1
2
1
1
1
0
2
0
3
2
1
1
2
